### **Generating all possible solutions for [Mathler](https://mathler) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

First of all, I thought this wouldn't be feasable, that the amount of possible equations and solutions would be to large to compute, it turns out that both **Easy Mode** and **Medium Mode**, fit on 32GB of RAM *(even when converting `itertools.product(*pattern)` to a `list`)*.

But, **Hard Mode** didn't fit as a `list`, so first, I changed the approach to a 'lazy evaluation'. Where equations and solutions would be written to the `.csv` file one at a time, instead of storing the whole dataset im RAM and then writing it to the `.csv`.

A last attempt was made using `pandas`, but it was significantly slower than the 'lazy evaluation' method *(6 times slower)*.

Let's list the explicit rules for each possible solution:
+ Each equation must be exactly 6 chars long *(each char is considered to be a single digit number or an operator)*
+ Numbers and operators can appear more than once
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parentheses
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Equations can start with a `'-'` sign, but can't start with any other operator *(`'/','*','+'`)*
+ Equations can't end with any of the operators *(`'/','*','+','-'`)*
+ Only divisions that result in an integer are valid
+ Divisions by zero are not allowed
+ There can't be equations that are equal to the solution:
  + 6 digit numbers
  + 5 digit negative numbers

Besides all of that, there are 3 modes:

+ **Easy Mathler** *(5 squares and 1 operator)*
+ **Mathler** *(6 squares and 2 operators)*
+ **Hard Mathler** *(8 squares and 3 operators)*

In [158]:
from tqdm import tqdm
import itertools
import csv
import re
import os

Defining constants:

In [159]:
EMPTY_STRING = ""  # An empty string
FILE_EXTENSION = ".csv" # The file extension for the output files
CSV_HEADER = ["equation", "solution"] # The header for the CSV file

NUMBERS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]  # List of valid numbers
OPERATORS = ["/", "*", "+", "-"] # List of valid operators
NEGATIVE_OPERATORS = ["-"]  # '-' operator

FIRST_CHAR = NUMBERS + NEGATIVE_OPERATORS # First character of the equation
MIDDLE_CHAR = NUMBERS + OPERATORS # Middle characters of the equation
LAST_CHAR = NUMBERS # Last character of the equation

DIFFICULTIES = {  # Stores all the dicciculty modes data
    "easy": {
        "basename": "equations_easy",
        "operators_count": 1,
        "pattern": [FIRST_CHAR] + [MIDDLE_CHAR] * 3 + [LAST_CHAR],
    },
    "medium": {
        "basename": "equations_medium",
        "operators_count": 2,
        "pattern": [FIRST_CHAR] + [MIDDLE_CHAR] * 4 + [LAST_CHAR],
    },
    "hard": {
        "basename": "equations_hard",
        "operators_count": 3,
        "pattern": [FIRST_CHAR] + [MIDDLE_CHAR] * 6 + [LAST_CHAR],
    },
}

Define more constants based on wheter we are running locally or on **Kaggle**

In [160]:
# Set parameters if running in Kaggle
if "kaggle" in os.getcwd():
    DIRPATH = "/kaggle/working/"
    SPLIT_FILES = False

# Set parameters if running locally
else:
    DIRPATH = "data\\"
    SPLIT_FILES = True
    MAX_FILE_SIZE = 99 * (1024 ** 2)  # 99MB

Defining helper functions:

In [161]:
def delete_previous_files(mode):
    """Deletes previous csv files for a given mode"""

    # Extract variable from DIFFICULTIES
    basename = DIFFICULTIES[mode]["basename"]

    # Scan the directory and delete all files containing the basename
    for dir_entry in os.scandir(DIRPATH):
        if dir_entry.is_file() and basename in dir_entry.path:
            os.remove(dir_entry.path)

def append_csv(filepath, row):
    """Appends a row to a csv file"""

    with open(filepath, "a", newline="") as file:
        csv.writer(file).writerow(row)

Defining the equation solver function:

In [162]:
def solve(input_equation):
    """Solves an equation passed as a string"""

    # Join all consecutive numbers
    equation = []
    temp = EMPTY_STRING

    for char in input_equation:
        if char in NUMBERS:
            temp += char

        else:
            if temp != EMPTY_STRING:
                equation.append(temp)

            equation.append(char)
            temp = EMPTY_STRING

    equation.append(temp)

    # If first element is a negative operator, join it to the next element (number)
    if equation[0] == NEGATIVE_OPERATORS[0]:
        equation[1] = f"{NEGATIVE_OPERATORS[0]}{equation[1]}"
        equation.pop(0)

    # Perform all operations (following the operators order - PEDMAS)
    for i in range(0, 3, 2):
        operators = OPERATORS[i : i + 2]

        while operators[0] in equation or operators[1] in equation:
            for j, _ in enumerate(equation):

                if equation[j] in operators:
                    operator = equation[j]

                    result = eval(f"{equation[j - 1]}{operator}{equation[j + 1]}")

                    if result != int(result):
                        return None

                    equation[j - 1] = int(result)

                    for _ in range(2):
                        equation.pop(j)

                    break

    # Return equations first element
    return equation[0]

Testing the solver:

In [163]:
# Define test cases
test_equations = [
    ("-3+4/2", -1),
    ("-3*4/2", -6),
    ("-10+10", 0),
    ("10-1-9", 0),
    ("40/4/2", 5),
    ("1+2+30", 33),
    ("3+21/3", 10),
    ("40/4+2", 12),
    ("16+4*2", 24),
    ("2/1*40", 80),
    ("1+2/30", None),
    ("2/3*10", None),
]

# Iterate over equations and check if they are solved correctly
for equation, solution in test_equations:
    assert solve(equation) == solution, f"{equation} should be {solution}"
else:
    print("All tests passed!")

All tests passed!


Defining the function that generates all of the valid equations for a given difficulty level:

In [164]:
def gen_equations(difficulty):
    """Generates all valid equations for the game"""

    # Define variables
    valid_equations_counter = 0
    file_number = 1

    # Extract variables and store them
    operators_count = DIFFICULTIES[difficulty]["operators_count"]
    pattern = DIFFICULTIES[difficulty]["pattern"]

    # Calculate total number of candidate equations
    candidates_count = 1
    for subpattern in pattern:
        candidates_count *= len(subpattern)

    # Combinations of elements for each position of the equation
    equations = itertools.product(*pattern)

    # Delete previous files
    delete_previous_files(difficulty)

    # Iterate over each equation
    for equations in tqdm(equations, total=candidates_count, desc=f"{difficulty.title()} equations"):

        # Convert equation to a single string
        equation = "".join(equations)

        # Count number of operators in an equation
        counter = 0
        for char in equation:
            if char in OPERATORS:
                counter += 1

        if equation[0] == NEGATIVE_OPERATORS[0]:
            counter -= 1

        if counter != operators_count:
            continue

        # Discard equations containing consecutive operators
        if re.findall(r"[/,*,+,-][/,*,+,-]", equation):
            continue
        
        # Discard equations containing leading zeros
        if re.findall(r"0\d", equation):
            continue

        # Discard equations containing divisions by zero
        if re.findall(r"/0", equation):
            continue

        # Solve the equation
        solution = solve(equation)

        # Discard equations with invalid solutions
        if not solution:
            continue
        
        #Discard equations that are equal to the solution
        if equation == solution:
            continue
        
        # Check if the solver's solution matches the eval()'s solution
        assert int(eval(equation)) == solution, f"The solver's solution is different than the eval()'s solution: {equation} != {solution}"
        
        # Check wheter output file should be split
        if SPLIT_FILES:

            # Build a temporary filepath
            filepath = f"{DIRPATH}{DIFFICULTIES[difficulty]['basename']}_{file_number:03}{FILE_EXTENSION}"

            # If file already exists, check wheter it's size is over the limit
            if os.path.exists(filepath):
                if os.path.getsize(filepath) >= MAX_FILE_SIZE:

                    # Increment file number counter and build a new filepath
                    file_number += 1
                    filepath = f"{DIRPATH}{DIFFICULTIES[difficulty]['basename']}_{file_number:03}{FILE_EXTENSION}"

                    # Init CSV file with header
                    append_csv(filepath, CSV_HEADER)
                
        else:
            # Build a temporary filepath
            filepath = f"{DIRPATH}{DIFFICULTIES[difficulty]['basename']}{FILE_EXTENSION}"

        # Append equation and solution to CSV file
        append_csv(filepath, [equation, solution])

        # Increment valid equations counter
        valid_equations_counter += 1
    
    return valid_equations_counter

Generating all valid equations for each difficulty mode:

In [165]:
for difficulty in DIFFICULTIES.keys():
    valid_equations_count = gen_equations(difficulty)

    print(f"Valid equations for {difficulty.upper()}:\t{valid_equations_count:,}")

Easy equations: 100%|██████████| 301840/301840 [00:31<00:00, 9703.38it/s] 


Valid equations for EASY:	79,145


Medium equations: 100%|██████████| 4225760/4225760 [03:01<00:00, 23302.33it/s] 


Valid equations for MEDIUM:	283,925


Hard equations:   3%|▎         | 26389687/828248960 [03:24<6:15:15, 35613.24it/s] 